In [2]:
import torch
import numpy as np
from PIL import Image
import torchvision
import cn_clip
import cn_clip.clip as clip
from cn_clip.clip import load_from_name, available_models, load

print("Available models:", available_models())
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
# model, preprocess = load_from_name("ViT-B-16", device=device, download_root='./pt_models')
model, preprocess = load_from_name("ViT-H-14", device=device, download_root='../02_CLIP/pt_models')  # 
model.eval()
print('finished')


from PIL import Image
from io import BytesIO
from urllib import request
def url2pil(img_url):
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36'
    headers = {'user-agent': user_agent}
    req = request.Request(url=img_url, headers=headers)
    response = request.urlopen(req, timeout=30)
    img= Image.open(BytesIO(response.read())).convert('RGB')
    return img

def get_img_emb(url):
    if not url: return ('1'+'***'+'url')
    try:
        img = url2pil(url)
    except:
        return ('2'+'***'+url)
    image = url2pil(url)
    with torch.no_grad():
        image = preprocess(image).unsqueeze(0).to(device)
        image_features = model.encode_image(image)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        image_features = image_features.cpu().numpy()
        emb = ','.join([str(x) for x in image_features.flatten()])
        return "***".join(('0',url, emb))
        
#get_img_emb(url)


def get_txt_emb_batch(text):
    text = clip.tokenize(text).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text)
    text_features = text_features.detach()
    text_features /= text_features.norm(dim=-1, keepdim=True)
    text_features = text_features.cpu().numpy()
    return text_features


Available models: ['ViT-B-16', 'ViT-L-14', 'ViT-L-14-336', 'ViT-H-14', 'RN50']
cpu
Loading vision model config from C:\Users\Aurora\AppData\Roaming\Python\Python310\site-packages\cn_clip\clip\model_configs\ViT-H-14.json
Loading text model config from C:\Users\Aurora\AppData\Roaming\Python\Python310\site-packages\cn_clip\clip\model_configs\RoBERTa-wwm-ext-large-chinese.json
Model info {'embed_dim': 1024, 'image_resolution': 224, 'vision_layers': 32, 'vision_width': 1280, 'vision_head_width': 80, 'vision_patch_size': 14, 'vocab_size': 21128, 'text_attention_probs_dropout_prob': 0.1, 'text_hidden_act': 'gelu', 'text_hidden_dropout_prob': 0.1, 'text_hidden_size': 1024, 'text_initializer_range': 0.02, 'text_intermediate_size': 4096, 'text_max_position_embeddings': 512, 'text_num_attention_heads': 16, 'text_num_hidden_layers': 24, 'text_type_vocab_size': 2}
finished


In [ ]:
import torch
from PIL import Image
import cn_clip.clip as clip
from cn_clip.clip import load_from_name
import torch.nn as nn
import torch.optim as optim

# Load CLIP model and preprocessing
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = load_from_name("ViT-H-14", device=device, download_root='../02_CLIP/pt_models')
model.eval()

# Load text tokenizer
text_tokenizer = clip.tokenize

# Define a simple classifier model
class Classifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(input_size, num_classes)

    def forward(self, x):
        return self.fc(x)

# Calculate the input size for the classifier
text_feature_dim = model.text_projection.shape[1]  # Change this based on the correct attribute for text features
image_feature_dim = model.visual.output_dim  # Change this based on the correct attribute for image features
input_size = text_feature_dim + image_feature_dim
num_classes = 2

# Train and save the classifier
classifier = Classifier(input_size, num_classes)
classifier.to(device)

# Assuming you have your own training data and training loop here...

# Save the classifier weights
torch.save(classifier.state_dict(), "your_classifier.pth")

# Load the saved classifier
classifier = Classifier(input_size, num_classes)
classifier.to(device)
classifier.load_state_dict(torch.load("your_classifier.pth"))
classifier.eval()

# Get input from user
user_text = input("Enter a piece of text: ")
image_path = input("Enter the path to an image: ")

# Process text
text = clip.tokenize([user_text]).to(device)

# Process image
image = Image.open(image_path).convert('RGB')
image = preprocess(image).unsqueeze(0).to(device)

# Calculate features
with torch.no_grad():
    text_features = model.encode_text(text)
    image_features = model.encode_image(image)

# Concatenate text and image features
concatenated_features = torch.cat((text_features, image_features), dim=1)

# Pass features through classifier
with torch.no_grad():
    logits = classifier(concatenated_features)
    probs = torch.softmax(logits, dim=-1)
    match_probability = probs[0, 1].item()  # Probability of match

print("Match probability:", match_probability)
